# Generate CT slice video
Lung and abdomen windows with optional segmentations

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor, wait
from pathlib import Path

import arrow
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk

from tqdm import tqdm

from video_utils import create_mp4
from plot_utils import plot_contours, plot_text, plot_slice_full, get_slice_range, plot_and_save

import matplotlib
matplotlib.use('Agg')  # so figures r not cached

### Note
This code assumes there is 1 volume and 1 segmentation per subject. If there are more volumes per subject, keep only the one that's segmented (mentioned in the spreadsheet in the dataset)

In [ ]:
dataset_folder = Path("/home/liushifeng/Desktop/AutoPET Lymphoma/dataset C_lymphoma")
seg_folder     = Path("/home/liushifeng/Desktop/AutoPET Lymphoma/SEG_Lymphoma_C_Rifki-selected")
output_folder  = Path("../outputs")
frames_folder  = output_folder / "frames"
videos_folder  = output_folder / "videos"

ct_filename = "CT.nii.gz"
ct_paths = list(dataset_folder.glob(f"*/*/{ct_filename}"))

## Generate video
Done concurrently, set the appropriate workers based on CPU cores 

In [ ]:
max_workers = 16
exists = True
for i, ct_path in enumerate(ct_paths):
    if "2e97a9e5c2" in str(ct_path):  # process will start after this
        exists = False
        continue
    if exists:
        continue
    
    dataset_name = ct_path.parts[-4]
    patient = ct_path.parts[-3]
    study = ct_path.parts[-2]

    print(
        f"[{i+1:03d}/{len(ct_paths)}] "
        f"Processing: {patient}/{study}"
    )
    
    scan_hash = ct_path.parent.parent.stem.split("_")[1]
    seg_path = list(seg_folder.glob(f"*{scan_hash}*"))[0]
    
    # load data
    ct_img = sitk.ReadImage(ct_path)
    seg_img = sitk.ReadImage(seg_path)
    seg_img = sitk.Resample(seg_img, ct_img, sitk.Transform(), sitk.sitkNearestNeighbor, 0.0, seg_img.GetPixelID())

    ct_array = sitk.GetArrayFromImage(ct_img)
    seg_array = sitk.GetArrayFromImage(seg_img)

    slice_range, n_slices = get_slice_range(seg_array)
    
    # create frames folder
    video_name = f"{patient}__{study}"
    ct_frames_folder = frames_folder / dataset_name / video_name
    ct_frames_folder.mkdir(exist_ok=True, parents=True)

    print(f"Generating {n_slices} frames")
    # for i in tqdm(range(*slice_range)):
    #     args = ct_array, seg_array, i, slice_range, ct_frames_folder
    #     plot_and_save(args)
    with ThreadPoolExecutor(max_workers=max_workers) as e:
        futures = []
        for i in range(*slice_range):
            args = ct_array, seg_array, i, slice_range, ct_frames_folder
            futures.append(e.submit(plot_and_save, args))
        wait(futures)
        
    date_str = arrow.now().format("YYYY-MM-DD")
    output_path = videos_folder / dataset_name / f"{video_name}_video_{date_str}.mp4"
    print(f"Creating video: {output_path.name}")
    create_mp4(output_path, ct_frames_folder)
    print(f"Video saved as {output_path}")
    
    print("-"*30)

In [ ]:
# visualize a single slice
i = 1360
fig = plot_slice_full(ct_array, seg_array, i, slice_range)
fig.savefig(f"test2.jpg", bbox_inches='tight', pad_inches=0, dpi=180);

## Window and visualize

In [ ]:
# Select a slice for visualization (e.g., middle slice)
slice_idx = 1360

# Create overlay
# fig, ax = plt.subplots()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.imshow(lung_ct[slice_idx], cmap='gray')
# ax1.imshow(seg_data[slice_idx], alpha=0.3, cmap='Reds')
ax1.set_title('Lung Window with Segmentation Overlay')

ax2.imshow(abdomen_ct[slice_idx], cmap='gray')
# ax2.imshow(seg_data[slice_idx], alpha=0.3, cmap='Reds')
ax2.set_title('Abdomen Window with Segmentation Overlay')

plt.show()

---
# Misc Scripts
## Rename files

In [ ]:
folder = Path("/home/liushifeng/Documents/GitHub/ctseg/outputs/dataset C_lymphoma/")
folder_names = os.listdir(folder)

for n in tqdm(folder_names):
    new_name = n.replace("dataset C_lymphoma__", "").replace("dataset C_lymphoma-", "")
    src = folder / n
    dst = folder / new_name
    os.rename(src, dst)